In [0]:
%sql
#------------------ MERGE------------------------------
CREATE TABLE learning.test.categories_for_merge_updates like learning.test.categories_delta;
INSERT INTO learning.test.categories_for_merge_updates
values (2,'Condiments_update','Sweet and savory sauces, relishes, spreads, and seasonings'),
 (22,'new category','aaaa')
MERGE INTO learning.test.categories_delta 
USING learning.test.categories_for_merge_updates
ON learning.test.categories_delta.categoryID = learning.test.categories_for_merge_updates.categoryID
WHEN MATCHED THEN
  UPDATE SET
    categoryID = learning.test.categories_for_merge_updates.categoryID,
    categoryName = learning.test.categories_for_merge_updates.categoryName
WHEN NOT MATCHED
  THEN INSERT (
    categoryID,
    categoryName,
    description
  )
  VALUES (
    learning.test.categories_for_merge_updates.categoryID,
    learning.test.categories_for_merge_updates.categoryName,
    learning.test.categories_for_merge_updates.description
  )


In [0]:

%sql
INSERT INTO learning.test.categories_for_merge_updates
values (2,'Condiments_update2','Sweet and savory sauces, relishes, spreads, and seasonings'),
 (222,'new category222','bbbb')

delete from learning.test.categories_for_merge_updates where categoryName = 'Condiments_update2'


In [0]:
%python
from delta.tables import *
categories = DeltaTable.forName(spark, "learning.test.categories_delta")
categories_for_merge = DeltaTable.forName(spark, "learning.test.categories_for_merge_updates")

df_categories_merge_update = categories_for_merge.toDF()
#df_categories_merge_update.show()

categories.alias('categories').merge(df_categories_merge_update.alias('updates'),'categories.categoryID = updates.categoryID' ) \
    .whenMatchedUpdate(set = {"categoryName" : "updates.categoryName"}) \
    .whenNotMatchedInsert(values = {
        "categoryID" : "updates.categoryID",
        "categoryName" : "updates.categoryName",
        "description" : "updates.description",
        }).execute()
 



In [0]:
Delete from learning.test.categories_for_merge_updates; 
INSERT INTO learning.test.categories_for_merge_updates values (112,'Condiments_update2','desc3');
 
MERGE INTO learning.test.categories_delta 
USING learning.test.categories_for_merge_updates
ON learning.test.categories_delta.categoryID = learning.test.categories_for_merge_updates.categoryID
WHEN NOT MATCHED BY SOURCE AND learning.test.categories_delta.categoryID < 3 THEN
  UPDATE SET learning.test.categories_delta.description = 'newww'


Learning timeintervals:  current_date() - INTERVAL '5' DAY

In [0]:
%sql
ALTER TABLE learning.test.categories_delta ADD COLUMN created_date TIMESTAMP;
DELETE FROM learning.test.categories_for_merge_updates;
INSERT INTO learning.test.categories_for_merge_updates values (1111,'Condiments_update2','desc3');

MERGE INTO learning.test.categories_delta 
USING learning.test.categories_for_merge_updates
ON learning.test.categories_delta.categoryID = learning.test.categories_for_merge_updates.categoryID
WHEN NOT MATCHED BY SOURCE THEN
  UPDATE SET created_date = current_timestamp() - Interval '3' day;


UPDATE learning.test.categories_delta set created_date = current_timestamp() - INTERVAL '33' day where categoryID = 222;



In [0]:
%python
from delta.tables import *
categories = DeltaTable.forName(spark, "learning.test.categories_delta")
categories_for_merge = DeltaTable.forName(spark, "learning.test.categories_for_merge_updates")
df_categories_merge_update = categories_for_merge.toDF()

categories.alias('target').merge(df_categories_merge_update.alias('source'), \
    'target.categoryID = source.categoryID') \
        .whenNotMatchedBySourceDelete(condition="target.created_date >= current_date() - INTERVAL 7 DAYS").execute()